In [1]:
import nltk; nltk.download('stopwords')
!python3 -m spacy download en
!pip install pyLDAvis
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 4.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 4.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=fc3b785ea272e749bc90017cd4ff0919ac

True

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
nlp = spacy.load("en_core_web_sm")

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [3]:
df = pd.read_csv('/content/drive/MyDrive/result.csv')
#df.drop('Unnamed:0')
df

,Unnamed: 0,id,speech,display_as,party,constituency,mnis_id,date,time,colnum,...,RE,Verb,RE2,Pres,topic,politics_phase,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords
0,0,uk.org.publicwhip/debate/1979-11-27a.1096.7,Is the Minister aware that in the last two mon...,Robert Maclennan,Labour,Caithness and Sutherland,578.0,1979-11-27,NaN,1096.0,...,"[('shells', 'vessels', 'Is'), ('they', 'last y...","[been, have, alarmed, landed, entered, appear,...","[('shells', 'vessels', 'Is'), ('they', 'last y...","['in the last two months', 'from my constituen...",NaN,"[[], [], []]",0,8.0,0.1397,"industry, company, make, market, british, busi..."
1,1,uk.org.publicwhip/debate/1979-11-27a.1136.2,Is not the right hon. Gentleman contradicting ...,Roger Moate,Conservative,Faversham,922.0,1979-11-27,16:40:00,1136.0,...,"[('', '', 'Is'), ('Gentleman', 'himself', 'con...","[contradicting, envisages, described, told, co...","[('', '', 'Is'), ('Gentleman', 'himself', 'con...",['on what he has described as the good routes'...,NaN,"[[], [], [], [], [], []]",1,0.0,0.1437,"local, service, area, authority, constituency,..."
2,2,uk.org.publicwhip/debate/1979-11-29a.1477.7,Does the right hon. Gentleman recall that he s...,James Lamond,Labour,Oldham East,894.0,1979-11-29,NaN,1477.0,...,"[('right hon', '', 'Does'), ('colour', 'Britis...","[Does, said, fishing, regarded, reconcile, pre...","[('right hon', '', 'Does'), ('colour', 'Britis...","['for immigrant votes', 'as equal', 'of colour...",NaN,"[[], [], [], []]",2,3.0,0.1612,"country, people, government, political, agree,..."
3,3,uk.org.publicwhip/debate/1979-11-12a.1080.3,"Question put, That the amendment be made:",Unknown,NaN,NaN,NaN,1979-11-12,00:30:00,1080.0,...,"[('', 'amendment', 'put')]","[put, made]","[('', 'amendment', 'put')]",[],NaN,[[]],3,5.0,0.1214,"bill, make, order, amendment, clause, power, l..."
4,4,uk.org.publicwhip/debate/1979-11-29a.1470.2,At a most recent census in August 1979 the Uni...,Alick Buchanan-Smith,Conservative,North Angus and Mearns,727.0,1979-11-29,NaN,1470.0,...,"[('breeding herd', 'pig', 'was')]",[],"[('breeding herd', 'pig', 'was')]","['At a most recent census in August 1979', 'in...",NaN,[[]],4,4.0,0.1172,"case, public, law, police, issue, individual, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,uk.org.publicwhip/debate/2021-02-24a.1024.2,"Motion made, and Question put forthwith (Stand...",Unknown,NaN,NaN,NaN,2021-02-24,18:56:00,1024.0,...,"[('Question', '83A(7', '\n '), ('following pro...","[made, put, Standing, following, apply, supple...","[('Question', '83A(7', '\n '), ('following pro...",['to the Telecommunications Infrastructure ( L...,1.0,"[[], ['Programme ', 'Programme ']]",9995,5.0,0.2410,"bill, make, order, amendment, clause, power, l..."
9996,9996,uk.org.publicwhip/debate/2021-04-28c.361.4,I have regular discussions with ministerial co...,David Duguid,Conservative,Banff and Buchan,4606.0,2021-04-28,NaN,361.0,...,"[('COP26', 'Scotland', 'have'), ('that', 'Unit...","[have, offers, devolved, brings, devolved, sup...","[('COP26', 'Scotland', 'have'), ('that', 'Unit...","['with ministerial colleagues', 'as well as', ...",NaN,"[[], []]",9996,3.0,0.1930,"country, people, government, political, agree,..."
9997,9997,uk.org.publicwhip/debate/2021-02-22b.655.7,I am really grateful to my hon. Friend for his...,Boris Johnson,Conservative,Uxbridge and South Ruislip,1423.0,2021-02-22,15:51:00,655.0,...,"[('I', 'hon', 'am'), ('', 'question', 'Friend'...","[support, suffered, distributed, put, focus, f...","[('I', 'hon', 'am'), ('', 'question', 'Friend'...","['to my hon', 'for his question', 'to their le...",NaN,"[[], [], [], [], [], []]",9997,9.0,0.1381,"work, important, friend, support, ensure, make..."
9998,9998,uk.org.publicwhip/debate/2021-02-04b.1186.0,I welcome this debate and thank my hon. Friend...,Robert Halfon,Conservative,Harlow,3985.0,2021-02-04,15:45:00,1186.0,...,"[('I', 'hon', 'welcome'), ('', 'town f

In [4]:
data = df.speech.values.tolist()

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
#speech 变成了列表，然后对列表进行一个处理
data_words = list(sent_to_words(data))

print(data_words[:1])

[['is', 'the', 'minister', 'aware', 'that', 'in', 'the', 'last', 'two', 'months', 'there', 'have', 'been', 'episodes', 'which', 'have', 'alarmed', 'fishermen', 'from', 'my', 'constituency', 'because', 'shells', 'have', 'landed', 'within', 'yards', 'of', 'their', 'vessels', 'is', 'he', 'aware', 'that', 'arrangements', 'entered', 'into', 'last', 'year', 'appear', 'not', 'to', 'be', 'working', 'as', 'they', 'might', 'will', 'he', 'examine', 'this', 'urgently']]


In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
#创建bigram 和trigram 模型，这个可以说明哪些词经常在一起出现使用
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
  
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#按顺序调用函数
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN'])

print(data_lemmatized[:1])

[['month', 'episode', 'fisherman', 'constituency', 'shell', 'yard', 'vessel', 'arrangement', 'year']]


In [8]:
#创建LDA model
#LDA主题模型的两个主要输入是字典（id2word）和语料库
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])
#上面的 (0, 1) 意味着，单词 id 0 在第一个文档中出现一次。同样，单词 id 1 出现两次，依此类推

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]]


In [11]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip -q mallet-2.0.8.zip
!mallet-2.0.8/bin/mallet

--2022-06-12 18:32:13--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip [following]
--2022-06-12 18:32:14--  https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  6.64MB/s    in 2.3s    

2022-06-12 18:32:17 (6.64 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]

Unrecognized command: 
Mallet 2.0 commands: 

  import-dir         load the contents of a directory into mallet instances (one per file)
  import-file        load a single file into mallet instances (

In [12]:

#Gensim 提供了一个包装器来从 Gensim 自身内部实现 Mallet 的 LDA。
#您只需要下载zip 文件，解压缩并在解压缩后的目录中提供 mallet 的路径即可。

#You should update this path as per the path of Mallet directory on your system.
mallet_path = 'mallet-2.0.8/bin/mallet'  



In [13]:
def trainmodel(topic):
  ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=topic, id2word=id2word)
  return ldamallet

In [14]:
from gensim.models.wrappers import ldamallet
# Show Topics ---some error with this 
ldamallet= trainmodel(17)
pprint(ldamallet.show_topics(num_words=10))
#output files
ldamallet.save("/content/drive/MyDrive/topic17/mallet_model.model")
new_model = gensim.models.ldamodel.LdaModel.load("/content/drive/MyDrive/topic17/mallet_model.model")

# # Compute Coherence Score
# coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_ldamallet = coherence_model_ldamallet.get_coherence()
# print('\nCoherence Score: ', coherence_ldamallet)

[(15,
  '0.058*"area" + 0.050*"authority" + 0.027*"housing" + 0.027*"council" + '
  '0.025*"constituency" + 0.024*"problem" + 0.022*"community" + 0.018*"people" '
  '+ 0.018*"house" + 0.018*"water"'),
 (5,
  '0.253*"friend" + 0.214*"member" + 0.064*"point" + 0.055*"debate" + '
  '0.022*"speech" + 0.021*"view" + 0.017*"fact" + 0.016*"issue" + '
  '0.013*"comment" + 0.012*"contribution"'),
 (2,
  '0.069*"year" + 0.043*"tax" + 0.038*"rate" + 0.035*"money" + '
  '0.033*"increase" + 0.029*"scheme" + 0.028*"cost" + 0.027*"figure" + '
  '0.025*"benefit" + 0.025*"income"'),
 (16,
  '0.036*"issue" + 0.032*"process" + 0.029*"agreement" + 0.025*"place" + '
  '0.024*"discussion" + 0.023*"matter" + 0.022*"action" + 0.021*"time" + '
  '0.021*"point" + 0.021*"lady"'),
 (11,
  '0.062*"school" + 0.039*"education" + 0.027*"child" + 0.026*"year" + '
  '0.023*"number" + 0.020*"training" + 0.018*"place" + 0.017*"level" + '
  '0.017*"system" + 0.016*"time"'),
 (9,
  '0.112*"service" + 0.037*"health" + 0.029

In [26]:
#准确率太低，弃用
# !pip install little-mallet-wrapper
# import little_mallet_wrapper as lmw
# output_directory_path = "/content/drive/MyDrive/topic17"
# num_topics = 17
# training_data = []
# for i in data_lemmatized:
#   y = " ".join(i)
#   training_data.append(y)
# training_data[:10]
# lmw.import_data(mallet_path,path_to_training_data,path_to_formatted_training_data,training_data)
# path_to_training_data           = output_directory_path + '/training.txt'
# path_to_formatted_training_data = output_directory_path + '/mallet.training'
# path_to_model                   = output_directory_path + '/mallet.model.' + str(num_topics)
# path_to_topic_keys              = output_directory_path + '/mallet.topic_keys.' + str(num_topics)
# path_to_topic_distributions     = output_directory_path + '/mallet.topic_distributions.' + str(num_topics)
# path_to_word_weights            = output_directory_path + '/mallet.word_weights.' + str(num_topics)
# path_to_diagnostics             = output_directory_path + '/mallet.diagnostics.' + str(num_topics) + '.xml'
# lmw.train_topic_model(mallet_path,
#                       path_to_formatted_training_data,
#                       path_to_model,
#                       path_to_topic_keys,
#                       path_to_topic_distributions,
#                       path_to_word_weights,
#                       path_to_diagnostics,
#                       num_topics)
# topic_keys = lmw.load_topic_keys(output_directory_path + '/mallet.topic_keys.' + str(num_topics))
# topic_distributions = lmw.load_topic_distributions(output_directory_path + '/mallet.topic_distributions.' + str(num_topics))
# topic_word_probability_dict = lmw.load_topic_word_distributions(output_directory_path + '/mallet.word_weights.' + str(num_topics))
# #words weight
# words_cloud =[]
# for _topic, _word_probability_dict in topic_word_probability_dict.items():
#     topic_words =[]
#     # print('Topic', _topic)
#     #topic_words.append([])
#     for _word, _probability in sorted(_word_probability_dict.items(), key=lambda x: x[1],reverse=True)[:20]:
#         #print(round(_probability, 4), '\t', _word)
#         #topic_words[1].append({'word':_word,'pre':_probability})
#         topic_words.append([_topic,_word,format(_probability,'.8f')])
        
#     words_cloud.append(topic_words)
# words_cloud

Training topic model...
Complete


In [13]:
from gensim.models.ldamodel import LdaModel
def convertldaMalletToldaGen(mallet_model):
    model_gensim = LdaModel(
        id2word=mallet_model.id2word, num_topics=mallet_model.num_topics,
        alpha=mallet_model.alpha) # original function has 'eta=0' argument
    model_gensim.state.sstats[...] = mallet_model.wordtopics
    model_gensim.sync_state()
    return model_gensim

ldagensim = convertldaMalletToldaGen(ldamallet)

In [14]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldagensim, corpus, id2word)
vis


/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
6     -0.173211  0.246892       1        1  6.484176
3     -0.039446 -0.080311       2        1  6.211427
4     -0.127789 -0.044147       3        1  6.183128
2     -0.138504  0.228583       4        1  6.138381
10    -0.051902 -0.067450       5        1  6.086717
15    -0.093203 -0.080222       6        1  6.080236
12     0.015398  0.199300       7        1  6.067774
0      0.031058 -0.076473       8        1  6.040431
5     -0.047727 -0.108116       9        1  5.912646
13    -0.076497  0.006860      10        1  5.868874
11    -0.008664 -0.114633      11        1  5.841683
8      0.275909  0.038893      12        1  5.735272
1     -0.119998  0.008795      13        1  5.684048
14    -0.003196 -0.195251      14        1  5.676426
9      0.011749 -0.098545      15        1  5.621262
7      0.312084  0.094680      16        1  5.346348
16     0.233939  0.041143      17        1  5.021173, topic_info=            Term         Freq        Total Category  logprob  loglift
123       member  5749.000000  5749.000000  Default  30.0000  30.0000
25        friend  3935.000000  3935.000000  Default  29.0000  29.0000
82        people  4603.000000  4603.000000  Default  28.0000  28.0000
44    government  6334.000000  6334.000000  Default  27.0000  27.0000
134      service  1934.000000  1934.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
1233        hour   119.266764   384.000394  Topic17  -4.8130   1.8222
432    committee   130.620205   643.981543  Topic17  -4.7220   1.3961
454     response    99.398241   328.727051  Topic17  -4.9952   1.7954
36      business   104.128844  1011.107027  Topic17  -4.9487   0.7183
4          month    99.398241   555.737837  Topic17  -4.9952   1.2703

[979 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
706      11  0.991329       abuse
2483     12  0.954196      accept
1665     16  0.957631  acceptance
1981      6  0.304980      access
1981      9  0.556662      access
...     ...       ...         ...
8         9  0.086788        year
8        13  0.130046        year
2036     13  0.975308   youngster
351      13  0.992435       youth
5415     15  0.978786        zone

[1617 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 5, 3, 11, 16, 13, 1, 6, 14, 12, 9, 2, 15, 10, 8, 17])

In [64]:
pyLDAvis.save_html(vis, '/content/drive/MyDrive/topic17/ldamallet.html')
ldamallet.print_topics(num_words=30)

[(0,
  '0.131*"people" + 0.075*"work" + 0.045*"benefit" + 0.041*"scheme" + 0.028*"government" + 0.025*"pension" + 0.021*"worker" + 0.020*"support" + 0.017*"system" + 0.017*"employment"'),
 (1,
  '0.060*"country" + 0.026*"people" + 0.020*"defence" + 0.019*"force" + 0.019*"water" + 0.018*"security" + 0.017*"man" + 0.017*"situation" + 0.017*"world" + 0.013*"event"'),
 (2,
  '0.097*"government" + 0.085*"year" + 0.044*"tax" + 0.040*"money" + 0.039*"rate" + 0.034*"increase" + 0.027*"figure" + 0.025*"council" + 0.025*"budget" + 0.022*"cost"'),
 (3,
  '0.077*"government" + 0.065*"people" + 0.052*"party" + 0.032*"issue" + 0.026*"election" + 0.023*"labour" + 0.023*"leader" + 0.022*"time" + 0.022*"vote" + 0.015*"thing"'),
 (4,
  '0.223*"friend" + 0.104*"question" + 0.088*"point" + 0.047*"matter" + 0.046*"issue" + 0.044*"statement" + 0.029*"answer" + 0.025*"concern" + 0.021*"lady" + 0.017*"view"'),
 (5,
  '0.055*"company" + 0.052*"business" + 0.025*"government" + 0.025*"price" + 0.022*"market" + 0

In [68]:
y = ldamallet.print_topics(num_words=20)
modified_format = []
for i in y:
  tmp = []
  tmp.append(i[0])
  tmp.append(i[1].split('+'))
  modified_format.append(tmp)
print(modified_format)
topic_keywords = []
for i in modified_format:
  for j in i[1]:
    topic_keywords_weight=[]
    topic_keywords_weight.append(i[0])
    y = j.split('*')
    topic_keywords_weight.append(float(y[0]))
    topic_keywords_weight.append(y[1].replace('"', ''))
    topic_keywords.append(topic_keywords_weight)

topic_keywords_weight=pd.DataFrame(topic_keywords, columns=['topic', 'fre','keywords'])    
topic_keywords_weight 
data = topic_keywords_weight.infer_objects()
data.to_csv('/content/drive/MyDrive/topic17/keywords_weight_20.csv')


[[0, ['0.131*"people" ', ' 0.075*"work" ', ' 0.045*"benefit" ', ' 0.041*"scheme" ', ' 0.028*"government" ', ' 0.025*"pension" ', ' 0.021*"worker" ', ' 0.020*"support" ', ' 0.017*"system" ', ' 0.017*"employment" ', ' 0.016*"number" ', ' 0.015*"job" ', ' 0.014*"change" ', ' 0.013*"employer" ', ' 0.012*"pay" ', ' 0.011*"group" ', ' 0.010*"pensioner" ', ' 0.010*"time" ', ' 0.010*"measure" ', ' 0.010*"allowance"']], [1, ['0.060*"country" ', ' 0.026*"people" ', ' 0.020*"defence" ', ' 0.019*"force" ', ' 0.019*"water" ', ' 0.018*"security" ', ' 0.017*"man" ', ' 0.017*"situation" ', ' 0.017*"world" ', ' 0.013*"event" ', ' 0.012*"action" ', ' 0.012*"house" ', ' 0.012*"end" ', ' 0.011*"coal" ', ' 0.011*"nation" ', ' 0.011*"part" ', ' 0.010*"day" ', ' 0.010*"today" ', ' 0.010*"year" ', ' 0.010*"hand"']], [2, ['0.097*"government" ', ' 0.085*"year" ', ' 0.044*"tax" ', ' 0.040*"money" ', ' 0.039*"rate" ', ' 0.034*"increase" ', ' 0.027*"figure" ', ' 0.025*"council" ', ' 0.025*"budget" ', ' 0.022*"cost

In [77]:
print(ldamallet[corpus[0]])
Topic_distribution = []
length = len(corpus)
for l in range(length):
  tmp = []
  tmp.append(l)
  y = ldamallet[corpus[l]]
  for i in y:
    tmp.append(i[1])
  Topic_distribution.append(tmp)

[(0, 0.04985044865403789), (1, 0.07433255788190983), (2, 0.05550016616816218), (3, 0.04985044865403789), (4, 0.051733687825412655), (5, 0.06868284036778553), (6, 0.06303312285366124), (7, 0.05361692699678742), (8, 0.04985044865403789), (9, 0.04985044865403789), (10, 0.04985044865403789), (11, 0.05550016616816218), (12, 0.08939847125290794), (13, 0.05926664451091171), (14, 0.06114988368228647), (15, 0.06114988368228647), (16, 0.057383405339536946)]


In [78]:
import pickle
with open('/content/drive/MyDrive/topic17/topic_distribution.pkl', 'wb') as f:
  pickle.dump(Topic_distribution, f)

In [60]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data_lemmatized)
#df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data_lemmatized)
# Format

df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.to_csv('/content/drive/MyDrive/topic17/df_dominant_topic.csv')
df_dominant_topic['keywords'][0]

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,12.0,0.0894,"proposal, change, regulation, agreement, part,...","[month, episode, fisherman, constituency, shel..."
1,1,16.0,0.1380,"system, line, transport, area, road, authority...","[contradicting, competition, route, competitio..."
2,2,3.0,0.0904,"government, people, party, issue, election, la...","[year, fishing, vote, citizen, colour, race, c..."
3,3,4.0,0.0758,"friend, question, point, matter, issue, statem...","[question, amendment]"
4,4,7.0,0.0758,"industry, policy, country, government, year, j...","[herd, gilt]"
...,...,...,...,...,...
9995,9995,14.0,0.1632,"bill, order, amendment, clause, legislation, c...","[motion, question, standing_order, provision, ..."
9996,9996,1.0,0.1152,"country, people, defence, force, water, securi...","[discussion, industry, stakeholder, opportunit..."
9997,9997,11.0,0.0863,"school, education, government, child, number, ...","[friend, question, support, pupil, loss, learn..."
9998,9998,6.0,0.2462,"community, area, development, problem, housing...","[friend, member, peterborough, government, yea..."


In [62]:
df_dominant_topic['Keywords'][0]

'proposal, change, regulation, agreement, part, point, process, view, interest, number'

In [ ]:
#tsen
hm = np.array([[y for (x,y) in ldamallet[corpus[i]]] for i in range(len(corpus))])
from sklearn.manifold import TSNE
tsne = TSNE(random_state=2017, perplexity=30, early_exaggeration=120)
embedding = tsne.fit_transform(hm)
embedding = pd.DataFrame(embedding, columns=['x','y'])
embedding['hue'] = hm.argmax(axis=1)
embedding.to_csv('/content/drive/MyDrive/topic17/embedding.csv')
embedding

In [ ]:
# #df = pd.read_csv('/content/drive/MyDrive/result.csv')
embedding = pd.read_csv('/content/drive/MyDrive/topic17/embedding.csv')
df_dominant_topic = pd.read_csv('/content/drive/MyDrive/topic17/df_dominant_topic.csv')

In [ ]:
df_topic_distribution = pd.DataFrame(topic_distributions, columns=['topic1', 'topic2', 'topic3','topic4', 'topic5', 'topic6','topic7', 'topic8', 'topic9','topic10', 'topic11','topic12', 'topic13', 'topic14','topic15','topic16','topic17'])

In [ ]:
df_topic_distribution

In [ ]:
df

In [ ]:
df_topic_distribution['x'] = embedding['x']
df_topic_distribution['y'] = embedding['y']
df_topic_distribution['ref'] = df['id']
df_topic_distribution['speech'] = df['speech']
df_topic_distribution['speaker'] = df['display_as']
df_topic_distribution['location'] = df['constituency']
df_topic_distribution['party'] =df['party']
df_topic_distribution['date'] = df['date']
import datetime
def clean_date(date):
  datem = datetime.datetime.strptime(date, "%Y-%m-%d")
  return datem.year
df_topic_distribution['clean_date'] = df_topic_distribution['date'].apply(clean_date)
df_topic_distribution

In [ ]:
result= pd.concat([df_topic_distribution, df_dominant_topic], axis=1)
result.to_csv('/content/drive/MyDrive/topic17/version.csv')
result